# Noita 'Rickroll' QR Code Noise Analysis From Scratch

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/probable-basilisk/noita_qr_analysis/blob/main/qr_noise_analysis.ipynb)


I have heard it said that nobody has been able to 'replicate' this analysis so here it is, in a conveniently
runnable colab notebook, totally from scratch starting with nothing more than the qrcode image from in game
and ending with both the binary and grayscale noise tables and the list of row offsets.

In [ ]:
# if you are running in colab, you'll need to run this cell to download the qrcode and moby dick
!wget -O qr.png https://github.com/probable-basilisk/noita_qr_analysis/blob/main/qr.png?raw=true
!wget -O mobydick.png https://github.com/probable-basilisk/noita_qr_analysis/blob/main/mobydick.png?raw=true

In [ ]:
# common imports
import numpy as np
import cv2
import math
import matplotlib.pyplot as plt
import itertools
import operator
import collections
import json
plt.rcParams['figure.figsize'] = [10, 5]

## Part 0: Data Prep / Isolating the Noise

In [ ]:
# figure out how to write this image as a png in a sensible way
def omniwrite(fn, img):
  img = np.copy(img.astype(np.uint8))
  if np.max(img) == 1:
    img *= 255
  if len(img.shape) < 3 or img.shape[2] == 1:
    img = np.dstack([img]*3)
  cv2.imwrite(fn, img)


In [ ]:
qrcode = cv2.imread("qr.png")
plt.imshow(qrcode)

In [ ]:
# First observations: the qrcode blocks are perfectly aligned to a 12x12 pixel grid
# This will simplify a lot of stuff
BLOCKSIZE = 12
IMAGEHEIGHT = qrcode.shape[0]
ROWSIZE = qrcode.shape[1]

def stack(img):
  return np.vstack([img[:,:,c] for c in range(3)])

def destack(stacked):
  return np.dstack([stacked[i*IMAGEHEIGHT:(i+1)*IMAGEHEIGHT] for i in range(3)])

def blockpos(row, col):
  return (row*BLOCKSIZE, col*BLOCKSIZE)

def extract_block(img, row, col, nrows=1, ncols=1):
  return img[row*BLOCKSIZE:(row+nrows)*BLOCKSIZE, col*BLOCKSIZE:(col+ncols)*BLOCKSIZE, :]

# Let's have a closer look at a white and black block
_ = plt.imshow(extract_block(qrcode, 5, 4, 1, 2))


In [ ]:
# Let's have a look at some histograms
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Grayscale Histograms For Entire Image, White and Black Blocks')

ax1.set_title("Entire Image")
_ = ax1.hist(qrcode.reshape((-1)), bins=32)

ax2.set_title("White Block")
_ = ax2.hist(extract_block(qrcode, 5, 4).reshape((-1)), bins=32)

ax3.set_title("Black Block")
_ = ax3.hist(extract_block(qrcode, 5, 5).reshape((-1)), bins=32)


#### Observations:

* a white block has a large number of 255s
* a black block a large number of 0s
* the histograms of white and black blocks are mirrors of each other.

#### Conclusion:

We can figure out if a block is 'white' or 'black' by just checking if it has
more 0s or 255s, and then we can turn white blocks into black blocks by
inverting the RGB values as `new_value = 255 - old_value`

In [ ]:
def clean_block(block):
  is_white = (np.sum(block == 255) > np.sum(block == 0))
  if is_white:
    return (1, 255 - block)
    #return (1, block ^ 0xFF)
  else:
    return (0, block)

def isolate_noise_from_qrcode(img):
  noise_only = np.zeros_like(img)
  qrcode_only = np.zeros(img.shape[:2]).astype(int)
  nrows = img.shape[0] // BLOCKSIZE
  ncols = img.shape[1] // BLOCKSIZE
  for row in range(nrows):
    for col in range(ncols):
      y, x = blockpos(row, col)
      (color, noise) = clean_block(extract_block(img, row, col))
      qrcode_only[y:y+BLOCKSIZE, x:x+BLOCKSIZE] = color
      noise_only[y:y+BLOCKSIZE, x:x+BLOCKSIZE, :] = noise
  return (qrcode_only, noise_only)

qrcode_clean, isolated_noise = isolate_noise_from_qrcode(qrcode)
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Separated QRCode and Noise')
ax1.set_title('Clean QR Code')
ax1.imshow(qrcode_clean)
ax2.set_title('Isolated Noise')
ax2.imshow(isolated_noise)
ax3.set_title('Histogram of Isolated Noise')
_ = ax3.hist(isolated_noise.reshape((-1)), bins=32)


Notice how we have now produced noise that has the histogram (distribution) of values in black blocks.

Our final step of data preparation is to split the noise into three images, one for each RGB channel, and produce a *binary*
version of the noise. As part of this prep, we also *XOR* the binary noise again with the clean QR code. 

Why? It makes the analysis work. 

The fact that you have to separately XOR the QRCode back into the binary noise is the reason
why you could reasonably say that there's a second level of rickroll.

In [ ]:
def prep_binary_noise(noise, clean_qr_code, double_xor):
  if double_xor:
    return np.logical_xor(noise > 0, clean_qr_code > 0)
  else:
    return noise > 0

def prep_noise(isolated_noise, clean_qr_code, double_xor):
  grayscale_noise = [isolated_noise[:, :, i] for i in range(3)]
  binary_noise = [prep_binary_noise(n, clean_qr_code, double_xor) for n in grayscale_noise]
  return (grayscale_noise, binary_noise)

def full_preprocess(src_image, double_xor):
  clean, noise = isolate_noise_from_qrcode(src_image)
  return prep_noise(noise, clean, double_xor)

(grayscale_noise, binary_noise) = prep_noise(isolated_noise, qrcode_clean, double_xor=True)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
fig.suptitle('Prepped Noise (Green Channel)')
ax1.set_title('Grayscale Noise (zoomed)')
ax1.imshow(grayscale_noise[1][0:100, 0:100])
ax2.set_title('Binary Noise (zoomed)')
_ = ax2.imshow(binary_noise[1][0:100, 0:100])


## Part 1: Extracting the Noise Table & Row Offsets

All the binary noise is actually drawn from a relatively small table of values: each row is simply a slice of a 4559 large noise table.
Here we extract that table *from scratch* starting with nothing more than the assumption that every row of binary noise
comes out of some larger noise sequence.

First, we're going to look for rows that have *exact overlaps* with other rows. To do this efficiently (i.e., without O(n^2) comparing
every row to every other row) we will build a hash table of every `MATCH_BITS` window of every row, and then check each row's
*tail* (the last `MATCH_BITS` of that row) against the hash table. Here I set `MATCH_BITS` to 32 so that the probability of accidentally
matching rows is so incredibly low that we can ignore the possibility.

In [ ]:
# first, dealing with three color channels separately is annoying, so
# we're going to just stack the color channels into one 3x tall image
binary_noise_stacked = np.vstack(binary_noise)
magnitude_noise_stacked = np.vstack([isolated_noise[:,:,c] for c in range(3)])

# How many bits of exact match we need to link two rows.
# Note that 32 bits of exact match means the likelihood of
# a false positive, assuming uniform random binary noise, is about
# one in four billion. In other words, we would have to get
# extraordinarily unlucky to accidentally match two rows by chance.
MATCH_BITS = 32

# Convenience for easily turning a binary array into an integer
PLACE_VALUES = 2 ** np.arange(MATCH_BITS)

Now, we build up the hash table and then prune it to produce a directed graph of overlapping rows.

In [ ]:
# Convert the last MATCH_BITS values of each row into an integer
# (just treat it as a binary number)
def calculate_row_tails(img):
  img = img.astype(int)
  return img[:, -MATCH_BITS:] @ PLACE_VALUES

# Create a dictionary that maps a *tail* (integer value)
# to the possible next rows that link to it
def create_possible_successors_map(noise):
  noise = noise.astype(int)
  successors = collections.defaultdict(set)
  # We iterate to width-1 because we do not want rows
  # to match with themselves: a successor needs to actually
  # *grow* the chain.
  for offset in range(noise.shape[1]-MATCH_BITS-1):
    windows = noise[:, offset:offset+MATCH_BITS]
    hashes = windows @ PLACE_VALUES
    for row, val in enumerate(hashes):
      successors[val].add((row, offset))
  return successors

# Count how many bits match between the tail of row_a
# and the candidate position pos in row_b
def count_alignment(row_a, row_b, pos):
  # =======|MATCH_BITS]        row_a
  #    [===|MATCH_BITS|======  row_b
  #        ^
  #        pos
  #    ~~~~~~~~~~~~~~~~ overlap
  overlap = pos + MATCH_BITS
  count = np.sum(row_a[-overlap:] == row_b[:overlap])
  return (count, overlap)

# Take the candidate successors and only keep the
# ones that actually exactly match in their overlaps
def prune_successors(candidates, rowidx, rows):
  successors = []
  for (candidx, pos) in candidates:
    (count, total) = count_alignment(rows[rowidx, :], rows[candidx, :], pos)
    if count == total:
      successors.append((candidx, pos, count))
  return successors

def create_successors(noise):
  succ = {}
  cand = create_possible_successors_map(noise)
  tails = calculate_row_tails(noise)
  for (rowidx, tail) in enumerate(tails):
    succ[rowidx] = prune_successors(cand[tail], rowidx, noise)
  return succ

succ = create_successors(binary_noise_stacked)


Next we search that graph to find the longest chain of overlapping rows.

In [ ]:
# we do this with memoization/dynamic programming:
# we keep a dictionary ('longests') that keeps track of
# the longest chain rooted at a given row.
def _longest_rooted_chain(root, succ, longests):
  if root in longests:
    return longests[root]
  # break out with special sentinel value if we hit a cycle
  longests[root] = ((0, 0), -1)
  max_length = ROWSIZE
  max_succ = (-1, -1)
  for (idx, pos, count) in succ[root]:
    (_, sublength) = _longest_rooted_chain(idx, succ, longests)
    if sublength == -1: # cycle!
      return (None, -1)
    length = ROWSIZE + (sublength - (pos + MATCH_BITS))
    if length > max_length:
      max_length = length
      max_succ = (idx, pos)
  (idx, pos) = max_succ
  longests[root] = ((idx, pos), max_length)
  return longests[root]

# Then to find the globally longest chain, we just
# try every possible starting point.
def longest_chain(succ):
  longests = {}
  max_chain = -1
  max_idx = -1
  for rowidx in succ.keys():
    (_, length) = _longest_rooted_chain(rowidx, succ, longests)
    if length == -1: # cycle!
      return -1, []
    if length > max_chain:
      max_chain = length
      max_idx = rowidx
  chain = [(max_idx, 0)]
  curidx = max_idx
  offset = 0
  while True:
    ((nextidx, pos), _) = longests[curidx]
    if nextidx < 0:
      break
    curidx = nextidx
    offset += ROWSIZE - (pos + MATCH_BITS)
    chain.append((curidx, offset))
  return max_chain, chain

def assemble_chain(noise, chain):
  chain_length = max(offset + ROWSIZE for (_, offset) in chain)
  res = np.zeros(chain_length).astype(noise.dtype)
  for (rowidx, offset) in chain:
    res[offset:offset+ROWSIZE] = noise[rowidx, :]
  return res

chainsize, chain = longest_chain(succ)
print("Found a chain of size:", chainsize)
print("Chain:", chain)


In [ ]:
initial_noisetable = assemble_chain(binary_noise_stacked, chain)
# if everything went correctly you should have gotten a noise table that's 4559 long,
# which is conveniently 47*97 so we can display it as rectangular image:
if chainsize == 4559:
  plt.imshow(initial_noisetable.reshape((47, 97)))

#### Finding The Row Offsets

Now, we find each noise row's position in the noise table by just trying every position and picking
the one with the maximum similarity. 

Rows do not always have a perfect match in the noise table: they can differ by a few pixels. But we'll
find that on average each row will have a >99% match to some location in the noise table.

In [ ]:
def find_row_offsets(noisetable, noise, normalize=True):
  offsets = []
  match_ratios = []
  # transform from [0, 1] -> [-1, 1]
  # (this is nicer for np.correlate)
  if normalize:
    table_normalized = noisetable.astype(float) * 2.0 - 1.0
    noise_normalized = noise.astype(float) * 2.0 - 1.0
  else:
    table_normalized = noisetable
    noise_normalized = noise
  rowsize = noise.shape[1]
  for (rowidx, row) in enumerate(noise):
    # we use np.correlate to find the position of maximum correlation
    # (this is vastly faster than for-looping ourselves)
    pos = np.argmax(np.correlate(
        table_normalized, noise_normalized[rowidx, :]))
    count = np.sum(row == noisetable[pos:pos+rowsize])
    offsets.append(pos)
    match_ratios.append(count / rowsize)
  return offsets, match_ratios

(row_offsets, match_ratios) = find_row_offsets(initial_noisetable, binary_noise_stacked)
print("Mean match ratio:", np.mean(match_ratios))


#### Refining the Noise Table

The noise table we have already is pretty good, but since rows can differ by a few pixels we can try to
get a better noise table by using the offsets we just calculated to figure out the value in the noise
table that is *most consistent* with the rows that overlap it.

In [ ]:
# Given a noise image and each row's offset into a noisetable,
# produce the noisetable that is *most consistent* with the image
def maximally_consistent_noisetable(offsets, noise):
  tablesize = np.max(offsets) + ROWSIZE

  # Place each row into the noise table and keep track of
  # how many 1s we see at each position, in addition to how many
  # rows 'see' that position in the table at all
  nones = np.zeros(tablesize)
  ncounts = np.zeros(tablesize)
  for row, offset in enumerate(offsets):
    nones[offset:offset+468] += noise[row, :]
    ncounts[offset:offset+468] += 1

  # the output noise table
  noisetable = np.zeros(tablesize)

  # set each noise table value to be the most common value
  # it has seen
  for idx in range(tablesize):
      none = nones[idx]
      ncount = ncounts[idx]
      if none > ncount / 2:
          noisetable[idx] = 1

  return noisetable

refined_noisetable = maximally_consistent_noisetable(
    row_offsets, binary_noise_stacked)

(refined_row_offsets, match_ratios) = find_row_offsets(
    refined_noisetable, binary_noise_stacked)
print("New mean match ratio:", np.mean(match_ratios))

if len(initial_noisetable) == 4559:
  fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))
  ax1.set_title('Initial Noisetable')
  ax1.imshow(initial_noisetable.reshape((47, 97)))
  ax2.set_title('Refined Noisetable')
  ax2.imshow(refined_noisetable.reshape((47, 97)))
  ax3.set_title('Difference')
  diff = initial_noisetable.astype(float) - refined_noisetable.astype(float)
  ax3.imshow(diff.reshape((47, 97)))


Note that refining the noise table changed only a few dozen pixels and very slightly improved the match ratio.

#### Reconstructing the Noise

Now, we can reconstruct the binary noise, and see how well it matches the binary noise we started with.

In [ ]:
def reconstruct(noisetable, offsets):
  return np.vstack([noisetable[o:o+ROWSIZE] for o in offsets])

reconstructed_noise = reconstruct(refined_noisetable, refined_row_offsets)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))
ax1.set_title('Original Noise (Green Channel)')
ax1.imshow(binary_noise_stacked[IMAGEHEIGHT:IMAGEHEIGHT*2,:])
ax2.set_title('Reconstructed Noise (Green Channel)')
ax2.imshow(reconstructed_noise[IMAGEHEIGHT:IMAGEHEIGHT*2,:])
ax3.set_title('Difference')
noise_diff = binary_noise_stacked.astype(float) - reconstructed_noise.astype(float)
ax3.imshow(np.abs(noise_diff[IMAGEHEIGHT:IMAGEHEIGHT*2,:]))

diff_pixels = np.sum(np.abs(noise_diff))
print(f"Total different pixels: {diff_pixels} / {noise_diff.size} ({100 * diff_pixels / noise_diff.size}%)")

### Recovering the Noise Magnitude

Now we need to recover the magnitude values of the noisetable in addition to the signs.

In [ ]:
def accum_magnitudes(half_magnitudes, offsets):
  maglists = collections.defaultdict(list)
  for rowidx, offset in enumerate(offsets):
    row = half_magnitudes[rowidx, :]
    for colidx in range(len(row)):
      val = row[colidx]
      if val > 0:
        maglists[offset + colidx].append(val)
  return maglists

def visualize_maglists(maglists):
  most_mags = max(len(v) for v in maglists.values())
  max_idx = max(maglists.keys())
  ret = np.zeros((most_mags, max_idx+1))
  for idx, mags in maglists.items():
    sorted_mags = sorted(mags)
    ret[:len(sorted_mags), idx] = np.array(sorted_mags)
  return ret

def collapse_mags(maglists, outsize):
  max_idx = max(maglists.keys())
  finalmags = np.zeros((outsize))
  ambigs = np.zeros((outsize))
  allzero = np.ones((outsize))
  for idx, mags in maglists.items():
    amags = np.array(mags)
    if len(amags) > 0:
      allzero[idx] = 0
      modals, counts = np.unique(amags, return_counts=True)
      modal = modals[np.argmax(counts)]
      minval = np.min(amags)
      finalmags[idx] = np.mean(amags) #modal
      if minval != modal: #np.max(amags):
        ambigs[idx] = 1
  return (finalmags, ambigs, allzero)

maglists = accum_magnitudes(magnitude_noise_stacked, refined_row_offsets)
(noise_magnitudes, ambig_magnitudes, allzero_magnitudes) = collapse_mags(maglists, len(refined_noisetable))

if len(noise_magnitudes) == 4559:
  fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))
  ax1.set_title('Noise Magnitudes')
  ax1.imshow(noise_magnitudes.reshape((47, 97)))
  ax2.set_title('Ambiguous Magnitudes')
  ax2.imshow(ambig_magnitudes.reshape((47, 97)))
  ax3.set_title('Magnitudes With no Data')
  ax3.imshow(allzero_magnitudes.reshape((47, 97)))

#magvis = visualize_maglists(maglists)
#omniwrite("magvis.png", magvis)


In [ ]:
_ = plt.hist(noise_magnitudes, bins=256)

In [ ]:
signed_noise = (refined_noisetable * 2 - 1) * noise_magnitudes
plt.imshow(signed_noise.reshape((47, 97)))

In [ ]:
_ = plt.hist(signed_noise, bins=128)
plt.title("Histogram of Signed Noise Table")

In [ ]:
magdeltas = []
for _, maglist in maglists.items():
  if len(maglist) < 10:
    continue
  ml = np.array(maglist) #.astype(float)
  modals, counts = np.unique(ml, return_counts=True)
  modal = modals[np.argmax(counts)]
  minval = np.min(ml)
  magdeltas.append(minval)
#magdeltas = np.concatenate(magdeltas)
_ = plt.hist(magdeltas, bins=256)

In [ ]:
magdeltas = []
for _, maglist in maglists.items():
  if len(maglist) == 0:
    continue
  ml = np.array(maglist) #.astype(float)
  modals, counts = np.unique(ml, return_counts=True)
  modal = modals[np.argmax(counts)]
  minval = np.min(ml)
  magdeltas.append(modal - minval)
#magdeltas = np.concatenate(magdeltas)
_ = plt.hist(magdeltas, bins=min(256, len(np.unique(magdeltas))))

In [ ]:
repro_signed_noise = reconstruct(signed_noise, refined_row_offsets)
qrstack = np.vstack([qrcode_clean*255]*3)
qrdirtied = qrstack.astype(float) + repro_signed_noise
qrdirtied = np.maximum(0.0, np.minimum(255, qrdirtied))
qrdirtied = qrdirtied #.astype(np.uint8)
qrrepro = qrdirtied #destack(qrdirtied)
reprodiff = qrrepro - stack(qrcode).astype(float)
cv2.imwrite("reproduced_qr_minval_noise.png", destack(qrrepro).astype(np.uint8))
plt.imshow(reprodiff.T)

In [ ]:
print("rmse error:", np.mean(reprodiff ** 2.0) ** 0.5)
print("mean abs diff:", np.mean(np.abs(reprodiff)))
reprovized = np.abs(reprodiff * 30.0)
omniwrite("hmmmmmmm.png", reprovized)
cv2.imwrite("magnified_reproduction_error.png", destack(reprovized).astype(np.uint8))
_ = plt.hist(reprodiff.reshape((-1)), bins=15)

In [ ]:
omniwrite("isolated_noise_stacked.png", stack(isolated_noise))

In [ ]:
just_mags = np.abs(repro_signed_noise)
omniwrite("reproduced_unsigned_mags_minval.png", just_mags)
masked = just_mags * (stack(isolated_noise) > 0)
just_mag_diff = np.abs(masked - stack(isolated_noise).astype(float))
omniwrite("reproduced_mag_diff_minval.png", just_mag_diff)
print("mean abs diff:", np.mean(just_mag_diff))

In [ ]:
signed_diff = masked - stack(isolated_noise).astype(float)
qrcode_clean_stack = np.vstack([qrcode_clean]*3)
modified_signed_diff = signed_diff * (qrcode_clean_stack * 2.0 - 1.0)

print(np.mean(signed_diff))
plt.imshow(signed_diff.T < 0)

In [ ]:
plt.plot(np.correlate(modified_signed_diff[210,:], signed_noise))

In [ ]:
(secondary_offsets, _) = find_row_offsets(signed_noise, signed_diff, False)

In [ ]:
print(secondary_offsets[0:10])
print(refined_row_offsets[0:10])

## Part 2: Cursory Observations of the Binary Noise Table

So the 4559 binary noise table itself looks very similar to uniform random binary data. The number of 0s and 1s is very balanced, so no surprises there.

In [ ]:
_ = plt.hist(refined_noisetable, bins=2)
plt.title("Histogram of Binary Noise Table")

Visually, qualitatively, the noisetable also looks very much like uniform random data.

In [ ]:
if len(refined_noisetable) == 4559:
  genuine_random_noisetable = np.random.randint(0, 2, refined_noisetable.shape)
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 5))
  fig.suptitle('Visual Comparison of Extracted Noisetable with Uniform Random Data')
  ax1.set_title('Extracted (Real) Noisetable')
  ax1.imshow(refined_noisetable.reshape((47, 97)))
  ax2.set_title('Uniform Random Noisetable')
  ax2.imshow(genuine_random_noisetable.reshape((47, 97)))

## Part 3: Initial Analysis of the Offsets

### Histograms and plots why not

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 5))
fig.suptitle('Plotting the Offsets')
ax1.set_title('Histogram of Offsets')
ax1.hist(refined_row_offsets, bins=100)
ax2.set_title('Plot of Offsets (Green Channel)')
ax2.plot(refined_row_offsets[468:468+468])

Nothing obviously stands out.


### Finding Repeated Sequences

Since we know every row's location in the noise table, we can now semi-efficiently look
for repeated sequences of noise (because they will consist of a repeated sequence of row offsets).

For this part, we are going to find every repeat that is at least two rows large: note that since
there are 4559 - 468 = 4091 possible offsets, the expectation of seeing a repeat of length two
by chance is roughly on the order of (468*3)/(4091^2) ~= one in ten thousand. So if the offsets were
generated uniformly at random, we really wouldn't expect to see even a single small repeat.

In [ ]:
def identical_run_length(seq, p0, p1):
  count = 0
  for a, b in zip(seq[p0:], seq[p1:]):
    if a != b:
      break
    count += 1
  return count

def find_repeated_offset_sequences(offsets):
  # first, build a map from offset -> row[]
  offset_positions = collections.defaultdict(list)
  for rowidx, offset in enumerate(offsets):
    offset_positions[offset].append(rowidx)

  runs = collections.defaultdict(int)
  runmatches = collections.defaultdict(list)

  for offset, positions in offset_positions.items():
    if len(positions) <= 1:
      # this offset only occurs once so can't repeat
      continue
    for row_a, row_b in itertools.combinations(positions, 2):
      runlen = identical_run_length(offsets, row_a, row_b)
      if runlen > 1:
        earlier_row = min(row_a, row_b)
        runs[earlier_row] = runlen
        runmatches[earlier_row].append(max(row_a, row_b))

  run_positions = sorted(runs.keys())
  for rowidx in run_positions:
    rl = runs[rowidx]
    if rl <= 1:
      continue
    for idx in range(1, rl+1):
      if runs[rowidx+idx] == rl-idx:
        runs[rowidx+idx] = 0

  ret = []
  for rowidx, run_length in runs.items():
    if run_length > 1:
      ret.append((run_length, rowidx, runmatches[rowidx][0]))
  return ret

def rowpos_to_imagepos(rowpos, channelnames=['B', 'G', 'R']):
  channel = channelnames[rowpos // IMAGEHEIGHT]
  ypos = rowpos % IMAGEHEIGHT
  return (channel, ypos)

def print_repeats(repeats, channelnames=['B', 'G', 'R']):
  for (repeat_length, row_a, row_b) in repeats:
    (ca, ya) = rowpos_to_imagepos(row_a, channelnames)
    (cb, yb) = rowpos_to_imagepos(row_b, channelnames)
    print(f"Repeat length {repeat_length} @ {ca}{ya}-{ya+repeat_length} matching {cb}{yb}-{yb+repeat_length}")

repeats = find_repeated_offset_sequences(refined_row_offsets)
print_repeats(repeats)

So, unlike the noisetable itself which doesn't have any obvious non-random characteristics, the
offsets into that noisetable have six repeats, five of which are so substantially large as to 
be effectively impossible by chance.

Visualizing the locations of the repeats as just solid colors:

In [ ]:
def vis_repeats(reps):
  # put more colors here if you need them
  COLORS = [100, 150, 200, 240, 50, 60, 70, 80] 

  dest = np.zeros((IMAGEHEIGHT*3, ROWSIZE), dtype=np.uint8)
  for color, (length, a, b) in zip(COLORS, reps):
    dest[a:a+length] = color
    dest[b:b+length] = color+10

  return dest

# This function isn't used at the moment
def crop_to_repeats(noise, offsets, repeats):
  out_img_parts = []
  out_offsets = []
  for (length, a, b) in repeats:
    out_offsets.extend(offsets[a:a+length])
    out_img_parts.append(noise[a:a+length,:])
  return (out_offsets, np.vstack(out_img_parts))

repeats_img = vis_repeats(repeats)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Locations of Repeated Rows of Noise')
ax1.set_title('Blue Channel')
ax1.imshow(repeats_img[:IMAGEHEIGHT,:])
ax2.set_title('Green Channel')
ax2.imshow(repeats_img[IMAGEHEIGHT:2*IMAGEHEIGHT,:])
ax3.set_title('Red Channel')
ax3.imshow(repeats_img[2*IMAGEHEIGHT:])

We can also search for not merely identical sequences (like 10, 30, 20 matching 10, 30, 20 elsewhere) but also 
sequences with additive shifts (e.g., 10, 30, 20 matching 15, 35, 25) by instead of directly searching the offsets,
searching the deltas or derivatives of the offsets (so 10, 30, 20 becomes 0, 20, -10).

In [ ]:
offset_arr = np.array(refined_row_offsets)
offset_deltas = offset_arr[1:] - offset_arr[:-1]
repeat_deltas = find_repeated_offset_sequences(offset_deltas)
print_repeats(repeat_deltas)

This finds only the repeats we already knew about (except the length 2 repeat because taking the delta reduces lengths by one). 

Given the strange mirror symmetry, what about mirrored (reversed) sequences? To search for that we can concatenate the offsets with their
reverse:

In [ ]:

offset_mirrored = np.hstack([offset_arr, np.flip(offset_arr)])
offset_mirrored_delta = offset_mirrored[1:] - offset_mirrored[:-1]
mirror_chans = ['B', 'G', 'R', '-R', '-G', '-B']

print("== Mirrored ==")
print_repeats(find_repeated_offset_sequences(offset_mirrored), mirror_chans)

print("== Mirrored Deltas ==")
print_repeats(find_repeated_offset_sequences(offset_mirrored_delta), mirror_chans)

Once again this just finds the same sequences we already knew about.

Out of the repeated sequences, the length 2 repeat is perhaps the most curious if only for its minimal size. Lets examine it in detail.

In [ ]:
(hmm_offsets, hmm_noise) = crop_to_repeats(binary_noise_stacked, refined_row_offsets, repeats)
print("min/max offset in repeat sections:", np.min(hmm_offsets), np.max(hmm_offsets))
coverage_map = np.zeros(refined_noisetable.shape)
for offset in hmm_offsets:
  coverage_map[offset:offset+1] += 1
  coverage_map[offset+ROWSIZE-1:offset+ROWSIZE] += 1
cmap = coverage_map.reshape((47,97))
plt.imshow(cmap)
#print(np.sum(coverage_map.reshape((47, 97)), axis=1))
omniwrite("cmap.png", 255.0*(cmap / np.max(cmap)))

In [ ]:
print(offset_arr[ROWSIZE*2+117:ROWSIZE*2+119])
print(offset_arr[ROWSIZE*2+466:ROWSIZE*2+ROWSIZE])
print(3317 + 1358)
print(3317 - 1358)

def stretch(data, n=32):
  return np.repeat(data.reshape((1, -1)), n, axis=0)

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(18, 5))
a = refined_noisetable[1358:1358+ROWSIZE]
b = refined_noisetable[3317:3317+ROWSIZE]
ax1.imshow(stretch(a))
ax2.imshow(stretch(b))
ax3.imshow(stretch(a.astype(np.uint8) ^ b.astype(np.uint8)))

In [ ]:
# Repeat length 66 @ R168-234 matching R234-300
r66_offsets = offset_arr[ROWSIZE*2+168:ROWSIZE*2+234]
r66_deltas = r66_offsets[1:] - r66_offsets[:-1]
print(r66_deltas)
M = 97
print(r66_offsets[:33] % M)
print(r66_offsets[33:] % M)
print((r66_offsets[33:] - r66_offsets[:33]) % M)
print((r66_offsets[33:] - np.flip(r66_offsets[:33])) % M)

def tobits(v, n=12):
  return [(v//(2**i))%2 for i in range(n)]

print(np.min(r66_offsets), np.max(r66_offsets))

print(tobits(3317))
print(tobits(1358))
print(np.array(tobits(3317)) ^ np.array(tobits(1358)))

#plt.scatter(r66_offsets[:33], np.flip(r66_offsets[33:]))

# tempo = (r66_offsets[33:] - r66_offsets[:33])

# tempimages = []
# for offsetoffset in range(4096 - 4041):
#   tempimages.append(np.vstack([np.array(tobits(o)) for o in (tempo + offsetoffset) % 1024]))
# plt.imshow(np.hstack(tempimages))
# omniwrite("offset_binary_shifts.png", np.hstack(tempimages))

In [ ]:
print_repeats(find_repeated_offset_sequences(secondary_offsets))

#### Observations

* There are no repeats in the blue channel.
* The red and green channels each have three repeats.
* The repeats in both channels are arranged in an ABCCAB pattern.
* Every repeat has either its start or end aligned to an exact quarter of the image height (multiple of 117).
* Every repeat has an even length (6 repeats, so about a 1/64th chance).

The number and size of the repeats rule out coincidence, while the arrangement of the repeats 
seems to bear the hallmarks of deliberate placement for some purpose.

The alignment of the repeats has a kind of mirror symmetry around the vertical center of the image,
but it's not clear why. Does this suggest that some kind of folding operation needs to be done?

## Part 4: Against The Malfunctioning PRNG Hypothesis

Initially I suspected that the curious structure of the noise could be due to poor
PRNG choice or usage. For example, LCGs (linear congruential generators) are notorious for producing
low-period output in their least significant bits, so that a careless programmer who reduces the range
of a random number by modulo (e.g., `rand() % 2` to produce a binary value) can end up with random values
with very short periods.

At this point we can see some immediate problems in this hypothesis:

* The noise table has a strange size (4559) and more importantly isn't a cycle
* A bad LCG would produce low-period output everywhere rather than irregularly sized repeated patches
* There are no repeated sections in the blue channel: a bad PRNG shouldn't be selective like this
* A bad PRNG wouldn't align itself to 4ths of the image unless its period happened to be "just so"

Nevertheless, it would be really convenient if we could declare this mystery solved as the result of
bad programmers picking a perversely bad PRNG, so let's not let mere arguments stop us. Instead, let
us pick a perversely bad PRNG and see what the machinery we have build does to it.

In [ ]:
# This is the LCG that Noita likes to use (MINSTD, a version of
# https://en.wikipedia.org/wiki/Lehmer_random_number_generator )

LCG_A = 7**5
LCG_M = (2**31 - 1)
def noita_lcg_next(v):
  return (LCG_A * v) % LCG_M

LCG_SEED = 0xd00df00d

# generate an image from a prng next function and a seed
def prng_image(w, h, prng_next, seed):
  res = np.zeros((w*h), dtype=np.int64)
  prng_state = seed
  for pos in range(w*h):
    prng_state = prng_next(prng_state)
    res[pos] = prng_state
  return res.reshape((h,w))

noita_lcg_noise_raw = prng_image(468, 468*3, noita_lcg_next, LCG_SEED)

# do the bad thing and just %2 it to get binary noise
noita_lcg_noise_mod = noita_lcg_noise_raw % 2

# do a slightly less bad thing and convert it to floats
noita_lcg_noise_float = noita_lcg_noise_raw.astype(np.float64) / LCG_M
noita_lcg_binary_float = (noita_lcg_noise_float < 0.5).astype(float)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
fig.suptitle("Noise produced by Noita's LCG (minstd)")
ax1.set_title('Mod 2')
ax1.imshow(noita_lcg_noise_mod[:100,:100])
ax2.set_title('Float scaled')
ax2.imshow(noita_lcg_binary_float[:100,:100])

In [ ]:
# now see if we can extract a noise chain of any sort
def test_noise(title, noise):
  print(title)
  succ = create_successors(noise)
  chainsize, chain = longest_chain(succ)
  if chainsize < 0:
    print("Cycle detected, but the logic to extract a cycle hasn't been written yet!")
    return
  print("Longest chain: ", chain)
  print("Chain length: ", chainsize)
  return chain

print("Note that it will always give you a chain of a single row (468)!\n")

# Run the real noise again just to convince ourselves our test actually works!
_ = test_noise("The actual QR code noise:", binary_noise_stacked)
print("")
_ = test_noise("Mod 2 MINSTD Noise:", noita_lcg_noise_mod)
print("")
_ = test_noise("Float-scaled MINSTD noise:", noita_lcg_binary_float)

Using the LCG that Noita actually uses in game did not result in any noise chains or cycles (note that the algorithm will always tell you
that there's a chain that's a single-row long, because every row can be considered a trivial chain by itself).

But, but... what if we make an even worse LCG.

In [ ]:
BAD_LCG_A = 83
BAD_LCG_M = (2**21 - 1) # try changing the exponent here
def incredibly_bad_lcg_next(v):
  return (BAD_LCG_A * v) % BAD_LCG_M

terrible_lcg_noise = prng_image(468, 468*3, incredibly_bad_lcg_next, 3) % 2
plt.imshow(terrible_lcg_noise[:100, :100])

In [ ]:
_ = test_noise("Terrible noise:", terrible_lcg_noise)

#### Discussion

Yes, you can make a really bad LCG that repeats. But the repeats are both very regular (because "periodic" means "periodic") and very visible.

It's hard to prove a negative: _could_ there be some PRNG that exhibits the characteristics of the QRCode noise? I can't rule it
out. But the problem is that:

* Noita already uses MINSTD
* MINSTD can be implemented in one line
* MINSTD is already considered a bad PRNG
* Every common PRNG in use these days is better than MINSTD
* Yet even MINSTD doesn't create noise that can be compressed into a tiny noise table!

So I think the idea that the Nolla programmers went out of their way, just for the QR code, to engineer
a special PRNG that's stranger and worse than the MINSTD they already use, and instead of using any of the
better PRNGs common available, is quite implausible.

## Part 5: Creating Our Own Noise

A reasonable question to ask is, 'how complicated is it to make such noise'?
If intentionally creating noise like in the qrcode is very difficult, then that
might be evidence that it is an unintended artifact of some other process. So 
lets try to reproduce it.

In [ ]:
# Pick how large we want our tables to be
# We don't have to to 4559!
SYNTH_TABLE_SIZE = 7777

# Create a signed noisetable
synth_noise_table = np.random.normal(0.0, 50.0, size=(SYNTH_TABLE_SIZE))

# Create random offsets
# (note: the actual qrcode offsets have "weird stuff" going on but we don't understand that)
synth_offsets = np.random.randint(0, SYNTH_TABLE_SIZE-ROWSIZE, size=IMAGEHEIGHT*3)

# Construct the base grayscale noise from gray table & offsets
synth_noise = np.vstack([synth_noise_table[o:o+ROWSIZE] for o in synth_offsets])

# Load in our 'secret' image and invert it into the noise
secret_image = (cv2.imread("mobydick.png")[:,:,0] > 0).astype(np.uint8)
secret_image = np.vstack([secret_image]*3) # for three channels
synth_noise[secret_image > 0] *= -1.0

# stack back into color image
synth_img = destack(synth_noise)

# add in visible QRCode and clamp and convert to uint8
for chan in range(3):
  synth_img[:,:,chan] += (qrcode_clean*255.0)
synth_img = np.maximum(0.0, np.minimum(255.0, synth_img))
synth_img = synth_img.astype(np.uint8)

plt.imshow(synth_img)
cv2.imwrite("moby_dick_qr.png", synth_img)

In [ ]:
# Now lets try solving our own image
def solve_image(img, signed_noise=True):
  (_, resynth_bin) = full_preprocess(img, double_xor=signed_noise)
  resynth_stacked = np.vstack(resynth_bin)
  resynth_chain = test_noise("Our own noise:", resynth_stacked)
  initial_noisetable = assemble_chain(resynth_stacked, resynth_chain)
  (row_offsets, match_ratios) = find_row_offsets(initial_noisetable, resynth_stacked)
  print("Mean match ratio:", np.mean(match_ratios))

  refined_noisetable = maximally_consistent_noisetable(
      row_offsets, resynth_stacked)

  (refined_row_offsets, match_ratios) = find_row_offsets(
      refined_noisetable, resynth_stacked)
  print("New mean match ratio:", np.mean(match_ratios))

  reco = reconstruct(refined_noisetable, refined_row_offsets)
  diff = reco - resynth_stacked
  return (reco, diff)

(reco_synth, diff_synth) = solve_image(synth_img, signed_noise=True)
plt.imshow(np.abs(diff_synth[468*1:468*2,:]))

In [ ]:
dd = np.abs(diff_synth)
cv2.imwrite("moby_dick_extracted.png", destack(dd.astype(np.uint8) * 255))